[exercises](brir.ipynb)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.io import loadmat
import soundfile as sf
import tools

In [ ]:
brir_clap, clap_fs = sf.read('data/brir_clap.wav')
clap_fs

In [ ]:
brir_clap.shape

In [ ]:
len(brir_clap) / clap_fs  # duration in seconds

In [ ]:
mat_contents = loadmat('data/brir_sweep.mat', struct_as_record=False, squeeze_me=True)
mat_contents['__header__']

In [ ]:
data = mat_contents['data']
type(data)

In [ ]:
data.SystemLatencySamples, data.SystemlatencyRemoved

In [ ]:
sweep_fs = data.fs
assert sweep_fs == clap_fs
sweep_fs

In [ ]:
data.head_azimuth

In [ ]:
brir_sweep = data.ir
brir_sweep.shape

In [ ]:
len(brir_sweep) / sweep_fs  # duration in seconds

In [ ]:
speech, speech_fs = sf.read('data/xmas.wav')
speech.shape

In [ ]:
assert speech_fs == clap_fs == sweep_fs
fs = speech_fs

In [ ]:
def convolve_multichannel_ir(x, h, **kwargs):
    """Convolve mono signal x with multichannel impulse reponse h.
    
    This is somewhat inefficient, because the FFT of x is repeated
    multiple times.
    
    After the convolution, the result is normalized to the maximum
    amplitude of x.    
    
    """
    x = np.squeeze(np.asarray(x))
    if x.ndim != 1:
        raise ValueError("x must be a mono signal")
        
    h = np.asarray(h)
    if h.ndim == 1:
        h = h.reshape(-1, 1)
        
    y = np.column_stack([signal.fftconvolve(x, ir, **kwargs) for ir in h.T])
    y = tools.normalize(y, np.max(np.abs(x)))
    return y

Note that `scipy.signal.fftconvolve()` does n-dimensional convolution.
Since we only need one-dimensional convolution, we have to call it with one-dimensional arrays.

Probably, some time in the future, `scipy.signal.fftconvolve()` will grow an `axis` argument ...

https://github.com/scipy/scipy/issues/3525

http://docs.scipy.org/doc/numpy/reference/generated/numpy.apply_along_axis.html

http://stackoverflow.com/a/29678671/500098

http://nipy.org/nitime/api/generated/nitime.utils.html#nitime.utils.fftconvolve

In [ ]:
speech_clap = convolve_multichannel_ir(speech, brir_clap)
speech_clap.shape

In [ ]:
speech_sweep = convolve_multichannel_ir(speech, brir_sweep)

In [ ]:
sf.write('data/xmas_brir_clap.wav', speech_clap, fs)
sf.write('data/xmas_brir_sweep.wav', speech_sweep, fs)

<audio src="data/xmas.wav" controls></audio>
[data/xmas.wav](data/xmas.wav)

<audio src="data/xmas_brir_clap.wav" controls></audio>
[data/xmas_brir_clap.wav](data/xmas_brir_clap.wav)

<audio src="data/xmas_brir_sweep.wav" controls></audio>
[data/xmas_brir_sweep.wav](data/xmas_brir_sweep.wav)


In [ ]:
hcomp, hcomp_fs = sf.read('data/THOMSON_HED415N_KEMAR_hcomp.wav')
assert hcomp_fs == fs
hcomp.shape

In [ ]:
def compensate(brir):
    """Apply headphone compensation to a BRIR.

    Side note: The variable "hcomp" is taken from the global scope
    (it is *not* a function argument)!

    """
    return np.column_stack([signal.fftconvolve(in1, in2)
                            for in1, in2 in zip(brir.T, hcomp.T)])

In [ ]:
brir_clap_hcomp = compensate(brir_clap)
brir_sweep_hcomp = compensate(brir_sweep)

In [ ]:
speech_clap_hcomp = convolve_multichannel_ir(speech, brir_clap_hcomp)
speech_sweep_hcomp = convolve_multichannel_ir(speech, brir_sweep_hcomp)

In [ ]:
sf.write('data/xmas_brir_clap_hcomp.wav', speech_clap_hcomp, fs)
sf.write('data/xmas_brir_sweep_hcomp.wav', speech_sweep_hcomp, fs)

<audio src="data/xmas.wav" controls></audio>
[data/xmas.wav](data/xmas.wav)

<audio src="data/xmas_brir_clap.wav" controls></audio>
[data/xmas_brir_clap.wav](data/xmas_brir_clap.wav)

<audio src="data/xmas_brir_clap_hcomp.wav" controls></audio>
[data/xmas_brir_clap_hcomp.wav](data/xmas_brir_clap_hcomp.wav)

---

<audio src="data/xmas.wav" controls></audio>
[data/xmas.wav](data/xmas.wav)

<audio src="data/xmas_brir_sweep.wav" controls></audio>
[data/xmas_brir_sweep.wav](data/xmas_brir_sweep.wav)

<audio src="data/xmas_brir_sweep_hcomp.wav" controls></audio>
[data/xmas_brir_sweep_hcomp.wav](data/xmas_brir_sweep_hcomp.wav)

---

Note that the headphone compensation filter used here was designed for THOMSON HED415N headphones.
If you use any other type of headphones, the result might not sound better, it might even sound worse!

In [ ]:
# plot BRIRs

plt.figure(figsize=(15, 10))
plt.subplot(221)
plt.plot(tools.db(tools.normalize(brir_clap)))
plt.title('BRIR clap')
plt.ylim(-200,0)

plt.subplot(222)
plt.plot(tools.db(tools.normalize(brir_clap_hcomp)))
plt.title('BRIR clap headphone-compensated')
plt.ylim(-200,0)

plt.subplot(223)
plt.plot(tools.db(tools.normalize(brir_sweep)))
plt.title('BRIR sweep')
plt.ylim(-200,0)

plt.subplot(224)
plt.plot(tools.db(tools.normalize(brir_sweep_hcomp)))
plt.title('BRIR sweep headphone-compensated')
plt.ylim(-200,0)

In [ ]:
# estimate time-of-flight
np.argmax(brir_clap)/clap_fs * 343

In [ ]:
# estimate SNR
print(tools.db(np.var(brir_clap[40000:]), power = True))
print(tools.db(np.var(brir_clap_hcomp[40000:]), power = True))
print(tools.db(np.var(brir_sweep[50000:]), power = True))
print(tools.db(np.var(brir_sweep_hcomp[50000:]), power = True))

In [ ]:
# plot magnitude spectrum
plt.figure(figsize=(12,5))

plt.subplot(121)
plt.plot(tools.db(tools.normalize(np.fft.rfft(brir_clap[:,0]))))
plt.plot(tools.db(tools.normalize(np.fft.rfft(brir_clap[:,1]))))
plt.xscale('log')
plt.title('BRIR clap');
plt.xlabel('f in Hz');
plt.ylabel('magnitude in dB');

plt.subplot(122)
plt.plot(tools.db(tools.normalize(np.fft.rfft(brir_sweep[:,0]))))
plt.plot(tools.db(tools.normalize(np.fft.rfft(brir_sweep[:,1]))))
plt.xscale('log')
plt.title('BRIR sweep');
plt.xlabel('f in Hz');
plt.ylabel('magnitude in dB');

In [ ]:
def load_ir_from_mat(angle):
    filename = 'data/brir_sweep{:+03d}.mat'.format(angle)
    data = loadmat(filename, struct_as_record=False, squeeze_me=True)['data']
    assert data.fs == fs
    assert np.isclose(np.rad2deg(data.head_azimuth), angle)
    return data.ir

In [ ]:
angles = -80, -40, 40, 80
brirs = {angle: load_ir_from_mat(angle) for angle in angles}

In [ ]:
brirs_hcomp = {angle: compensate(ir) for angle, ir in brirs.items()}

In [ ]:
speech_hcomp = {angle: convolve_multichannel_ir(speech, ir)
                for angle, ir in brirs_hcomp.items()}

<audio src="data/xmas.wav" controls></audio>
[data/xmas.wav](data/xmas.wav)

<audio src="data/xmas_brir_sweep_hcomp-80.wav" controls></audio>
[data/xmas_brir_sweep_hcomp-80.wav](data/xmas_brir_sweep_hcomp-80.wav)

<audio src="data/xmas_brir_sweep_hcomp-40.wav" controls></audio>
[data/xmas_brir_sweep_hcomp-40.wav](data/xmas_brir_sweep_hcomp-40.wav)

<audio src="data/xmas_brir_sweep_hcomp.wav" controls></audio>
[data/xmas_brir_sweep_hcomp.wav](data/xmas_brir_sweep_hcomp.wav)

<audio src="data/xmas_brir_sweep_hcomp+40.wav" controls></audio>
[data/xmas_brir_sweep_hcomp+40.wav](data/xmas_brir_sweep_hcomp+40.wav)

<audio src="data/xmas_brir_sweep_hcomp+80.wav" controls></audio>
[data/xmas_brir_sweep_hcomp+80.wav](data/xmas_brir_sweep_hcomp+80.wav)

In [ ]:
# plot impulse responses of 1 ear for all angles

time = np.arange(len(brir_sweep)) / sweep_fs

for angle in angles:
    plt.plot(time, brirs[angle][:,0])
    
plt.xlabel('time in s');

In [ ]:
# plot magnitude spectra

freq = np.arange(len(data.ir)/2 +1) * data.fs/(len(data.ir))

for angle in angles:
    plt.plot(freq, np.abs(np.fft.rfft(brirs[angle][:,0])), label = '%i' %angle)

plt.legend();
plt.xlabel('frequency in Hz');

In the top right is the angle of the loudspeaker, relative to the head.

In [ ]:
import io
from urllib.request import urlopen

url =  'https://dev.qu.tu-berlin.de/projects/measurements/repository/raw/2010-11-kemar-anechoic/mat/QU_KEMAR_anechoic_2m.mat'
file = io.BytesIO(urlopen(url).read())

# if you want to load a local file instead:
#file = "QU_KEMAR_anechoic_2m.mat"

In [ ]:
irs = loadmat(file, struct_as_record=False, squeeze_me=True)['irs']

In [ ]:
plt.imshow(irs.left);

What?

In [ ]:
plt.imshow(irs.left, aspect='auto');

Better, but still: what?

Let's try to convert the values to decibels:

In [ ]:
plt.imshow(20 * np.log10(np.abs(irs.left)), aspect='auto')
plt.colorbar();

OK, we're getting somewhere, but there are still things to do:

* replace x- and y-indices with meaningful values

* axis labels

* better colormap (the default colormap 'jet' is considered harmful, see e.g. https://jakevdp.github.io/blog/2014/10/16/how-bad-is-your-colormap/)

  * try those: 'YlGnBu', 'YlGn', 'RdPu', 'PuRd', 'PuBuGn', 'Greys'
  
  * if you have matplotlib >= 1.5, you should also try 'viridis', which will become the new standard color map in matplotlib 2.0

In [ ]:
def plot_irs(samples):
    extent = [np.rad2deg(irs.apparent_azimuth[0]),
              np.rad2deg(irs.apparent_azimuth[-1]),
              len(irs.left[:samples]) * 1000 / irs.fs,
              0]
    plt.imshow(20 * np.log10(np.abs(irs.left[:samples])), aspect='auto',
               vmax=0, vmin=-120,
               cmap='YlGnBu', extent=extent)
    plt.xlabel("apparent source azimuth / degrees")
    plt.ylabel("time / milliseconds")
    plt.colorbar()
    
plot_irs(-1)

In [ ]:
plot_irs(500)

In [ ]:
plot_irs(200)

In [ ]:
# TODO: waterfall diagrams?

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span rel="dct:publisher" resource="[_:publisher]">the person who associated CC0</span>
  with this work has waived all copyright and related or neighboring
  rights to this work.
</p>